In [1]:
!pip install opencv-python
!pip install dlib
!pip install pyautogui
!pip install pygame

In [ ]:
import cv2

cam = cv2.VideoCapture(0)

if not cam.isOpened():
    print("Error: Unable to access the camera.")
    exit()

cv2.namedWindow("Camera Feed")

while True:
    ret, frame = cam.read()

    if not ret:
        print("Error: Failed to retrieve a frame from the camera.")
        break

    cv2.imshow("Camera Feed", frame)

    if cv2.waitKey(1) == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()


In [ ]:
import dlib
import numpy as np

# Initialize the face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Initialize the array to store the data
data = []

# Collect the data
while True:
    # Read a frame from the camera
    ret, frame = cam.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = detector(gray)

    # Loop over the faces and detect facial landmarks
    for face in faces:
        landmarks = predictor(gray, face)

        # Extract the x, y coordinates of the facial landmarks
        coords = np.zeros((68, 2), dtype=int)
        for i in range(68):
            coords[i] = (landmarks.part(i).x, landmarks.part(i).y)

        # Append the coordinates to the data array
        data.append(coords)

        # Draw the facial landmarks on the frame
        for (x, y) in coords:
            cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)

    # Display the frame in the window
    cv2.imshow("Camera Feed", frame)

    # Press 'q' to exit the loop and stop collecting data
    if cv2.waitKey(1) == ord('q'):
        break

# Release the camera and destroy the window
cam.release()
cv2.destroyAllWindows()

# Convert the data array to a NumPy array and save it to a file
data = np.array(data)
np.save("data.npy", data)

In [ ]:
import pygame
import numpy as np

# Load the data from the file
data = np.load("data.npy")

# Initialize Pygame
pygame.init()

# Set the dimensions of the window
WINDOW_WIDTH = 640
WINDOW_HEIGHT = 480
screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))

# Set the dimensions of the avatar
AVATAR_WIDTH = 150
AVATAR_HEIGHT = 200

# Load the avatar image and scale it to the appropriate size
avatar = pygame.image.load("avatar.png")
avatar = pygame.transform.scale(avatar, (AVATAR_WIDTH, AVATAR_HEIGHT))

# Set the initial position of the avatar
x = WINDOW_WIDTH // 2 - AVATAR_WIDTH // 2
y = WINDOW_HEIGHT // 2 - AVATAR_HEIGHT // 2

# Set the maximum and minimum movement distances
MAX_MOVE = 50
MIN_MOVE = 5

# Set the movement speed
SPEED = 5

# Create a clock to control the frame rate
clock = pygame.time.Clock()

# Loop over the data and animate the avatar
for i in range(len(data)):
    # Clear the screen
    screen.fill((255, 255, 255))

    # Draw the avatar at the current position
    screen.blit(avatar, (x, y))

    # Calculate the movement distance
    dx = data[i][30][0] - data[i][27][0]
    dy = data[i][30][1] - data[i][27][1]
    distance = np.sqrt(dx**2 + dy**2)

    # Normalize the movement distance
    move = (distance - MIN_MOVE) / (MAX_MOVE - MIN_MOVE)
    move = min(max(move, 0), 1)

    # Calculate the new position of the avatar
    new_x = x + int(dx * move * SPEED)
    new_y = y + int(dy * move * SPEED)

    # Set the new position of the avatar
    x = min(max(new_x, 0), WINDOW_WIDTH - AVATAR_WIDTH)
    y = min(max(new_y, 0), WINDOW_HEIGHT - AVATAR_HEIGHT)

    # Update the display
    pygame.display.flip()

    # Control the frame rate
    clock.tick(30)

In [ ]:
import pygame
import numpy as np
import pyautogui

# Load the data from the file
data = np.load("data.npy")

# Initialize Pygame
pygame.init()

# Set the dimensions of the window
WINDOW_WIDTH = 640
WINDOW_HEIGHT = 480
screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))

# Set the dimensions of the avatar
AVATAR_WIDTH = 150
AVATAR_HEIGHT = 200

# Load the avatar image and scale it to the appropriate size
avatar = pygame.image.load("avatar.png")
avatar = pygame.transform.scale(avatar, (AVATAR_WIDTH, AVATAR_HEIGHT))

# Set the initial position of the avatar
x = WINDOW_WIDTH // 2 - AVATAR_WIDTH // 2
y = WINDOW_HEIGHT // 2 - AVATAR_HEIGHT // 2

# Set the maximum and minimum movement distances
MAX_MOVE = 50
MIN_MOVE = 5

# Set the movement speed
SPEED = 5

# Create a clock to control the frame rate
clock = pygame.time.Clock()

# Loop over the data and animate the avatar
for i in range(len(data)):
    # Clear the screen
    screen.fill((255, 255, 255))

    # Draw the avatar at the current position
    screen.blit(avatar, (x, y))

    # Calculate the movement distance
    dx = data[i][30][0] - data[i][27][0]
    dy = data[i][30][1] - data[i][27][1]
    distance = np.sqrt(dx**2 + dy**2)

    # Normalize the movement distance
    move = (distance - MIN_MOVE) / (MAX_MOVE - MIN_MOVE)
    move = min(max(move, 0), 1)

    # Calculate the new position of the avatar
    new_x = x + int(dx * move * SPEED)
    new_y = y + int(dy * move * SPEED)

    # Set the new position of the avatar
    x = min(max(new_x, 0), WINDOW_WIDTH - AVATAR_WIDTH)
    y = min(max(new_y, 0), WINDOW_HEIGHT - AVATAR_HEIGHT)

    # Update the display
    pygame.display.flip()

    # Control the frame rate
    clock.tick(30)

    # Get the facial expression
    smile_ratio = (data[i][51][1] - data[i][57][1]) / (data[i][50][0] - data[i][44][0])

    # Control the movement of the avatar based on the facial expression
    if smile_ratio > 0.5:
        pyautogui.moveRel(10, 0, duration=0.1)
    elif smile_ratio < 0.2:
        pyautogui.moveRel(-10, 0, duration=0.1)

In [1]:

"""
To do this, we need to run the script we just created. Before running the script, make sure
that you have all the necessary packages installed, including OpenCV, dlib, Pygame, NumPy,
and PyAutoGUI. Once you have installed the required packages, you can run the script using the following command:

python avatar.py

"""